## Scope
- See https://docs.julialang.org/en/v1/manual/variables-and-scoping/
- structrures with the `struct` keyword
- modules with the `module` keyword

In [1]:
data = [1, 2, 3]
s = 0
beta, gamma = 2, 1

for i in 1:length(data)
    print(i, " ")
    global s
    s += beta * data[i]
end

println("\nSum of data in external scope is $s")

1 2 3 
Sum of data in external scope is 12


In [2]:
# bubble sort
function bubbleSort!(a)
    n = length(a)
    for i in 1:n-1
        for j in 1:n-i
            if a[j] > a[j+1]
                a[j], a[j+1] = a[j+1], a[j]
            end
        end
    end
    return a
end

bubbleSort! (generic function with 1 method)

In [5]:
a = [3, 2, 1]
bubbleSort!(a)

3-element Vector{Int64}:
 1
 2
 3

### Roots of a polynomial

Consider the polynomial $f(x) = a_nx^n + a_n-1x^n-1 + ... + a_1x + a_0$

In [8]:
using Roots

In [31]:
# The .. is a splat operator
# This means that the function arguments are variadic
# Simplest example of a variadic function is print, since it takes arbitrary number arguments separated by a comma
function polynomialGenerator(a...)
    n = length(a) - 1
    poly = function(x) 
        return sum(a[i+1] * x^i for i in 0:n)
    end
    return poly
end
    

polynomialGenerator (generic function with 1 method)

In [32]:
polynomial = polynomialGenerator(1, 3, -10)
zeroVals = find_zeros(polynomial, -10, 10)
println("Zeros are $zeroVals")

Zeros are [-0.19999999999999998, 0.5]
